# 8. 树

## 8.1 树的嵌套列表实现
用递归的嵌套列表实现一个二叉树。

```[root, left, right]```: root表示根节点（值），left和right分别表示左右子树，也是一个列表。

In [2]:
myTree = ['a',  # 树根
          ['b', # 左子树
           ['d', [], []],
           ['e', [], []]],
          ['c', # 右子树
           ['f', [], []],
           ['g', [], []]]]

In [4]:
def BinaryTree(r):  # 创建一个二叉树 只有根节点
    return [r, [], []]

def insertLeft(root, newBranch):  # 在左子树插入新节点
    t = root.pop(1)  # 移除索引1的元素（左子树），同时返回这个元素
    if len(t) > 1:  # 判断原左子树是否为空
        root.insert(1, [newBranch, t, []])  # 新节点插入到索引1的位置，原左子树作为新节点的左子树
    else:
        root.insert(1, [newBranch, [], []])  # 新节点作为左子树 ，原左子树为空  
    return root

def insertRight(root, newBranch):  # 在右子树插入新节点
    t = root.pop(2)  # 弹出右子树
    if len(t) > 1:  # 如果右子树不为空
        root.insert(2, [newBranch, [], t])  # 新节点作为右子树，原右子树作为新节点的右子树
    else:
        root.insert(2, [newBranch, [], []])  # 新节点作为右子树，原右子树为空
    return root

def getRootVal(root):  # 获取根节点的值
    return root[0]  # 根节点的值在索引0处

def setRootVal(root, newVal):  # 设置根节点的值
    root[0] = newVal    # 将新值赋给索引0处

def getLeftChild(root):  # 获取左子树
    return root[1]  # 左子树在索引1处

def getRightChild(root):  # 获取右子树
    return root[2]

r = BinaryTree(3)   # 创建一个根节点为3的二叉树
insertLeft(r, 4)    # 在左子树插入4
insertLeft(r, 5)    # 在左子树插入5
insertRight(r, 6)   # 在右子树插入6
insertRight(r, 7)   # 在右子树插入7
print(r)  # [3, [5, [4, [], []], []], [7, [], [6, [], []]]]
print(getRootVal(r))  # 3
setRootVal(r, 9)
print(getRootVal(r))  # 9
print(getLeftChild(r))  # [5, [4, [], []], []]
print(getRightChild(r))  # [7, [], [6, [], []]]

[3, [5, [4, [], []], []], [7, [], [6, [], []]]]
3
9
[5, [4, [], []], []]
[7, [], [6, [], []]]


## 8.2 树的链表实现
用节点链接法来实现树，每个节点保存根节点的数据项，以及指向左右子树的链接。

In [ ]:
class BinaryTree:
    def __init__(self, rootObj):
        self.key = rootObj  # 根节点数据项
        self.leftChild = None   
        self.rightChild = None

    def insertLeft(self, newNode):
        if self.leftChild is None:  # 如果左子树为空
            self.leftChild = BinaryTree(newNode)  # 直接插入新节点
        else:   # 左子树不为空 类似于链表插入
            t = BinaryTree(newNode)  # 创建新节点
            t.leftChild = self.leftChild  # 将原左子树作为新节点的左子树
            self.leftChild = t  # 新节点作为当前节点的左子树
    
    def insertRight(self, newNode):
        if self.rightChild is None:  # 如果右子树为空
            self.rightChild = BinaryTree(newNode)  # 直接插入新节点
        else:
            t = BinaryTree(newNode)  # 创建新节点
            t.rightChild = self.rightChild  # 将原右子树作为新节点的右子树
            self.rightChild = t  # 新节点作为当前节点的右子树

    def getRightChild(self):
        return self.rightChild  # 获取右子树
    def getLeftChild(self):
        return self.leftChild   # 获取左子树
    def setRootVal(self, obj):
        self.key = obj  # 设置根节点的值
    def getRootVal(self):
        return self.key # 获取根节点的值

r = BinaryTree(3)   # 创建一个根节点为3的二叉树
r.insertLeft(4)    # 在左子树插入4
r.insertLeft(5)    # 在左子树插入5
r.insertRight(6)   # 在右子树插入6
r.insertRight(7)   # 在右子树插入7
print(r.getRootVal())  # 3
print(r.getLeftChild().getRootVal())  # 5
print(r.getRightChild().getRootVal())  # 7
print(r.getLeftChild().getLeftChild().getRootVal())  # 4
print(r.getRightChild().getRightChild().getRootVal())  # 6

3
5
7
4
6


## 8.3 表达式解析
1. 从全括号表达式构建树；
2. 利用表达式解析树求值；
3. 从表达式解析树恢复原始的字符串表达式；

步骤：
1. 全括号表达式分解为token：分为括号、操作符、操作数几类；
2. 读取：初始化当前节点（根节点），读取左括号则当前节点下降，右括号则上升；读取到操作符，则设置当前节点值为操作符，并新建右节点；读取操作数，则将当前节点值设为操作数，当前节点上升到父节点。

思路：
1. 创建左右子树用insertLeft/Right；
2. 当前节点设置用setRootVal；
3. 下降用getLeft/Right；
4. 上升节点用一个栈来记录跟踪父节点：节点下降时，将下降前的节点push入栈；需要上升时直接pop出栈的节点即可。

In [10]:
# step1 创建表达式解析树
def buildParseTree(fpexp):
    fplist = fpexp.split()  # 将表达式字符串拆分为列表
    pStack = []  # 创建一个栈用于存储父节点
    eTree = BinaryTree('')  # 创建一个空的二叉树
    pStack.append(eTree)  # 将树压入栈中
    currentTree = eTree  # 当前树指向eTree

    for i in fplist:
        if i == '(':  # 遇到左括号，创建左子树
            currentTree.insertLeft('')  # 插入一个空节点作为左子树
            pStack.append(currentTree)  # 将当前树压入栈中
            currentTree = currentTree.getLeftChild()  # 移动到左子树

        elif i not in ['+', '-', '*', '/', ')']:  # 遇到操作数
            currentTree.setRootVal(int(i))  # 设置当前节点的值为操作数
            parent = pStack.pop()  # 弹出栈顶元素作为父节点
            currentTree = parent  # 当前树指向父节点 出栈上升
    
        elif i in ['+', '-', '*', '/']:  # 遇到运算符
            currentTree.setRootVal(i)  # 设置当前节点的值为运算符
            currentTree.insertRight('')  # 插入一个空节点作为右子树
            pStack.append(currentTree)  # 将当前树压入栈中
            currentTree = currentTree.getRightChild()  # 移动到右子树

        elif i == ')':  # 遇到右括号，完成当前子树的构建
            currentTree = pStack.pop()  # 弹出栈顶元素作为当前树 出栈上升
        else:
            raise ValueError("Unknown Operator: " + i)
    return eTree
pt = buildParseTree("( ( 10 + 5 ) * 3 )")
print(pt.getRootVal())  # *
print(pt.getLeftChild().getRootVal())  # +
print(pt.getLeftChild().getLeftChild().getRootVal())  # 10
print(pt.getLeftChild().getRightChild().getRootVal())  # 5
print(pt.getRightChild().getRootVal())  # 3

*
+
10
5
3


In [ ]:
# step2 计算表达式解析树的值
import operator
def evaluate(parseTree):
    opers = {'+': operator.add, '-': operator.sub,
             '*': operator.mul, '/': operator.truediv}  # 定义运算符对应的函数
    leftC = parseTree.getLeftChild()  # 获取左子树
    rightC = parseTree.getRightChild()  # 获取右子树

    if leftC and rightC:  # 如果有左子树和右子树
        fn = opers[parseTree.getRootVal()]  # 获取当前节点的运算符对应的函数
        return fn(evaluate(leftC), evaluate(rightC))  # 递归计算左子树和右子树的值，并应用运算符函数
    else:
        return parseTree.getRootVal()  # 如果是叶节点，返回节点的值
print(evaluate(pt))  # 45

45
